In [7]:
import base64
import pandas as pd
import io
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
from dash.dependencies import State

In [4]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pile_module #custom module

# Define app layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Pile Information"),
    dropdown,
    html.Div(id='pile-info'),

    html.H1("File Upload"),
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        multiple=False
    ),
    html.Div(id='output-data-upload'),
])

# Callback to display pile info
@app.callback(
    Output('pile-info', 'children'),
    [Input('pile-dropdown', 'value')]
)
def display_pile_info(selected_value):
    return pile_module.display_row(selected_value, H_Pile)

# Callback to handle file upload
@app.callback(Output('output-data-upload', 'children'),
              [Input('upload-data', 'contents')],
              [State('upload-data', 'filename')])
def update_output(contents, filename):
    return pile_module.update_output(contents, filename)

if __name__ == '__main__':
    app.run_server(debug=True)